In [1]:
import scipy
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

for m in [scipy, np, pd, matplotlib]:
    print(m.__name__, m.__version__)

from scipy.signal import savgol_filter
from scipy.signal import argrelextrema
from scipy.signal import find_peaks

from funcs.factor_utils import FactorUtil

scipy 1.15.2
numpy 1.26.4
pandas 2.2.3
matplotlib 3.10.1


In [2]:
bar_df_1 = FactorUtil.get_bar_df('FG2409_MIN_1', '20240813', '00:00:00', 200)
bar_df_1.head()

,O,H,L,C,V,AP,AV,BP,BV
ts,,,,,,,,,
2024-08-13 09:00:00,1278.0,1278.0,1276.0,1276.0,681,1278.000,218,1276.000,152
2024-08-13 09:01:00,1277.0,1282.0,1276.0,1278.0,5557,1279.737,21421,1278.345,17282
2024-08-13 09:02:00,1278.0,1279.0,1276.0,1278.0,1634,1278.102,43934,1277.119,31489
2024-08-13 09:03:00,1277.0,1282.0,1277.0,1281.0,3749,1280.911,33442,1279.997,33766
2024-08-13 09:04:00,1281.0,1281.0,1279.0,1280.0,1619,1280.149,41884,1279.158,46279


In [3]:
s_close = savgol_filter(bar_df_1['C'], window_length=5, polyorder=2)
bar_df_1['S'] = s_close

In [4]:
peaks, _ = find_peaks(s_close, height=10, distance=10, prominence=5)
troughs, _ = find_peaks(-s_close, height=10, distance=10, prominence=5)
peaks, troughs

(array([ 38, 161, 172]), array([], dtype=int64))

In [5]:
def build_peaks(df):
    s = savgol_filter(df['C'], window_length=5, polyorder=2)
    df['S'] = s
    # print(df.head())
    peaks, _ = find_peaks(s, height=10, distance=10, prominence=5)
    troughs, _ = find_peaks(-s, height=10, distance=10, prominence=5)
    df.assign(P=np.nan)
    for idx in peaks:
        print(idx)
        df_idx = df['S'].iloc[idx].index
        print(df['S'].iloc[idx])
        df.loc[df_idx, 'P'] = df['S'].iloc[idx]
    

In [7]:
bar_df_1.head()

,O,H,L,C,V,AP,AV,BP,BV,S
ts,,,,,,,,,,
2024-08-13 09:00:00,1278.0,1278.0,1276.0,1276.0,681,1278.000,218,1276.000,152,1275.971429
2024-08-13 09:01:00,1277.0,1282.0,1276.0,1278.0,5557,1279.737,21421,1278.345,17282,1277.714286
2024-08-13 09:02:00,1278.0,1279.0,1276.0,1278.0,1634,1278.102,43934,1277.119,31489,1279.028571
2024-08-13 09:03:00,1277.0,1282.0,1277.0,1281.0,3749,1280.911,33442,1279.997,33766,1279.800000
2024-08-13 09:04:00,1281.0,1281.0,1279.0,1280.0,1619,1280.149,41884,1279.158,46279,1281.028571


In [9]:
bar_df_1.assign(P = np.nan)
for idx in peaks:
    df_idx = bar_df_1['S'].iloc[idx].index
    print(bar_df_1['S'].iloc[idx])
    bar_df_1.loc[df_idx, 'P'] = bar_df_1['S'].iloc[idx]

AttributeError: 'numpy.float64' object has no attribute 'index'

In [ ]:
ax = plt.plot(bar_df_1['C'])

In [ ]:
plt.plot(bar_df_1['C'])
plt.plot(peaks, bar_df_1['C'][peaks], "x", label="Peaks")
plt.plot(troughs, bar_df_1['C'][troughs], "o", label="Troughs")
plt.legend()
plt.show()